# Pinterest Data Pipeline
This is for the README File in the future. May contain additional notes for myself wherever I have problems.

#### Key Pair
Creating the keypar was our first course of action using the provided Account ID, IAM user name and Password to connect to the AWS Account and navigate through the console to locate the appropriate **KeyPairId**.
</br>
See file [0e2a0bfcc015-key-pair.pem](0e2a0bfcc015-key-pair.pem)

#### Connect to EC2
ssh -i "0e2a0bfcc015-key-pair.pem" ec2-user@ec2-184-73-75-230.compute-1.amazonaws.com

In [ ]:
ssh -i "0e2a0bfcc015-key-pair.pem" ec2-user@ec2-184-73-75-230.compute-1.amazonaws.com

Then ran code 

In [ ]:
sudo yum update

You can then use the standard command line commands to navigate around the file directory. <br>
Also, you can use "nano" to create and open files.

SET UP CLIENT PROPERTIES INSIDE KAFKA

In [ ]:

# Sets up TLS for encryption and SASL for authN.
security.protocol = SASL_SSL

#Identifies the ASL# S mechanism to use.
sasl.mechanism = AWS_MSK_IAM

# Binds SASL client implementation.
sasl.jaas.config = software.amazon.msk.auth.iam.IAMLoginModule required awsRoleArn="arn:aws:iam::584739742957:role/0e2a0bfcc015-ec2-access-role";

# Encapsulates constructing a SigV4 signature based on extracted credentials.
# The SASL client bound by "sasl.jaas.config" invokes this class.
sasl.client.callback.handler.class = software.amazon.msk.auth.iam.IAMClientCallbackHandler

#### Setting Up Kafka
first ran: sudo yum update
<br> then installed Java using: sudo yum install java-1.8.0-openjdk.x86_64 java-1.8.0-openjdk-devel.x86_64

<br> Kafka Target
<br> https://archive.apache.org/dist/kafka/2.8.1/kafka_2.12-2.8.1.tgz
</br>
May consider following, https://www.digitalocean.com/community/tutorials/how-to-install-apache-kafka-on-ubuntu-20-04


In [ ]:
wget https://archive.apache.org/dist/kafka/2.8.1/kafka_2.12-2.8.1.tgz

#### Created CLIENT PROPERTIES file inside of the EC2 > Kafka

In [ ]:
b-2.pinterestmskcluster.w8g8jt.c12.kafka.us-east-1.amazonaws.com:9098,b-1.pinterestmskcluster.w8g8jt.c12.kafka.us-east-1.amazonaws.com:9098,b-3.pinterestmskcluster.w8g8jt.c12.kafka.us-east-1.amazonaws.com:9098

#### Plaintext Apache Zookeeper connection string

In [ ]:
z-2.pinterestmskcluster.w8g8jt.c12.kafka.us-east-1.amazonaws.com:2181,z-1.pinterestmskcluster.w8g8jt.c12.kafka.us-east-1.amazonaws.com:2181,z-3.pinterestmskcluster.w8g8jt.c12.kafka.us-east-1.amazonaws.com:2181

#### Create Kafka Topic
To do this, while located inside the Kafka_Folder/bin use the below code.

In [ ]:
./kafka-topics.sh --bootstrap-server BootstrapServerString --command-config client.properties --create --topic <topic_name>

replacing the BootstrapServerString for the previously listed BOOTSTRAP SERVER PRIVATE ENDPOINT.

PINTEREST POST DATA

In [ ]:
./kafka-topics.sh --bootstrap-server b-2.pinterestmskcluster.w8g8jt.c12.kafka.us-east-1.amazonaws.com:9098,b-1.pinterestmskcluster.w8g8jt.c12.kafka.us-east-1.amazonaws.com:9098,b-3.pinterestmskcluster.w8g8jt.c12.kafka.us-east-1.amazonaws.com:9098 --command-config client.properties --create --topic 0e2a0bfcc015.pin

POST GEO LOCATION

In [ ]:
./kafka-topics.sh --bootstrap-server b-2.pinterestmskcluster.w8g8jt.c12.kafka.us-east-1.amazonaws.com:9098,b-1.pinterestmskcluster.w8g8jt.c12.kafka.us-east-1.amazonaws.com:9098,b-3.pinterestmskcluster.w8g8jt.c12.kafka.us-east-1.amazonaws.com:9098 --command-config client.properties --create --topic 0e2a0bfcc015.geo

POST USER DATA

In [ ]:
./kafka-topics.sh --bootstrap-server b-2.pinterestmskcluster.w8g8jt.c12.kafka.us-east-1.amazonaws.com:9098,b-1.pinterestmskcluster.w8g8jt.c12.kafka.us-east-1.amazonaws.com:9098,b-3.pinterestmskcluster.w8g8jt.c12.kafka.us-east-1.amazonaws.com:9098 --command-config client.properties --create --topic 0e2a0bfcc015.user

Kafka TOPICS Created.

0e2a0bfcc015.pin
0e2a0bfcc015.geo
0e2a0bfcc015.user

## M3: Configure EC2 Kafka Client

## M4: Connect MSK Cluster to S3
Using Confluent.io Amazon S3 Connector we will link it with our S3 bucket. This will allow data that flows through our Kafka Stream to be stored in our S3 Bucket for larger periods of time to allow for detailed analysis.

```
# assume admin user privileges
sudo -u ec2-user -i
# create directory where we will save our connector 
mkdir kafka-connect-s3 && cd kafka-connect-s3
# download connector from Confluent
wget https://d1i4a15mxbxib1.cloudfront.net/api/plugins/confluentinc/kafka-connect-s3/versions/10.0.3/confluentinc-kafka-connect-s3-10.0.3.zip
# copy connector to our S3 bucket
aws s3 cp ./confluentinc-kafka-connect-s3-10.0.3.zip s3://<BUCKET_NAME>/kafka-connect-s3/
```

After that's done.\
You can simply create a **"CUSTOM PLUGIN"** via the Amazon MSK interface.\
Locate the CONFLUENT connector zip\
Give the plugin a name and details\
CREATE PLUGIN \

Next, create a connector\
Select the PLUGIN you created\
Give it a name & detail\
Choose your MSK Cluster\
Then drop the following into the CONNECTOR CONFIGURATION SETTING

```
connector.class=io.confluent.connect.s3.S3SinkConnector
# same region as our bucket and cluster
s3.region=us-east-1
flush.size=1
schema.compatibility=NONE
tasks.max=3
# include nomeclature of topic name, given here as an example will read all data from topic names starting with msk.topic....
topics.regex=<YOUR_UUID>.*
format.class=io.confluent.connect.s3.format.json.JsonFormat
partitioner.class=io.confluent.connect.storage.partitioner.DefaultPartitioner
value.converter.schemas.enable=false
value.converter=org.apache.kafka.connect.json.JsonConverter
storage.class=io.confluent.connect.s3.storage.S3Storage
key.converter=org.apache.kafka.connect.storage.StringConverter
s3.bucket.name=<BUCKET_NAME>
```

Finally\
Change CONNECTOR TYPE into PROVISIONED\
Select CUSTOM CONFIGURATION for the WORKER CONFIGURATION - then select "confluent-worker"\
Select IAM role for ACCESS PERMISSION\

and DONE.


## M5: Configure API Gateway
To Replicate the Pinterest's experimental data pipeline we will need to build our own API. This API will send data to the MSK cluster, which in turn will be stored in an S3 bucket, using the connector we have built in the previous milestone.

##### T1: Build the Kafka REST proxy API

1 - Go to the API Gateway Console on the MSK. API Gateway > APIs \
2 - (since for this project API was already made, we will find it using our user-id) \
3 - Create a {proxy+} resource \
4 - Next setup the INTEGRATION through the EDIT INTEGRATION and select HTTP proxy \
5 - For the ENDPOINT URL, use the Public IPv4 DNS found in the MSK console for EC2 instances. (Format should be, http://```**KafkaClientEC2InstancePublicDNS**```:8082/{proxy}) \
6 - Finally you are set to DEPLOY the API Gateway
7 - Provide a stage Name and details.

invoke URL Created: https://moyj7yazp4.execute-api.us-east-1.amazonaws.com/pinDP



##### T2: Setup Kafka REST proxy on the EC2

1 - First install confluent package. \
```sudo wget https://packages.confluent.io/archive/7.2/confluent-7.2.0.tar.gz ``` \
```tar -xvzf confluent-7.2.0.tar.gz```

2 - Next open up the Kafka REST properties, located inside the confluent install. \ 
```confluent-7.2.0/etc/kafka-rest```

3 - Insert the following into the Kafka REST Properties \
```
#Sets up TLS for encryption and SASL for authN.
client.security.protocol = SASL_SSL

#Identifies the SASL mechanism to use.
client.sasl.mechanism = AWS_MSK_IAM

#Binds SASL client implementation.
client.sasl.jaas.config = software.amazon.msk.auth.iam.IAMLoginModule required awsRoleArn="Your Access Role";

#Encapsulates constructing a SigV4 signature based on extracted credentials.
#The SASL client bound by "sasl.jaas.config" invokes this class.
client.sasl.client.callback.handler.class = software.amazon.msk.auth.iam.IAMClientCallbackHandler

```

In [ ]:
from user_posting_emulation import *
run_infinite_post_data_loop()

SAMPLE OUTPUT

```
{'index': 7528, 'unique_id': 'fbe53c66-3442-4773-b19e-d3ec6f54dddf', 'title': 'No Title Data Available', 'description': 'No description available Story format', 'poster_name': 'User Info Error', 'follower_count': 'User Info Error', 'tag_list': 'N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e', 'is_image_or_video': 'multi-video(story page format)', 'image_src': 'Image src error.', 'downloaded': 0, 'save_location': 'Local save in /data/mens-fashion', 'category': 'mens-fashion'}

{'ind': 7528, 'timestamp': datetime.datetime(2020, 8, 28, 3, 52, 47), 'latitude': -89.9787, 'longitude': -173.293, 'country': 'Albania'}

{'ind': 7528, 'first_name': 'Abigail', 'last_name': 'Ali', 'age': 20, 'date_joined': datetime.datetime(2015, 10, 24, 11, 23, 51)}
```

KAFKA CONSUMER

```
./kafka-console-consumer.sh --bootstrap-server b-3.pinterestmskcluster.w8g8jt.c12.kafka.us-east-1.amazonaws.com:9098,b-2.pinterestmskcluster.w8g8jt.c12.kafka.us-east-1.amazonaws.com:9098,b-1.pinterestmskcluster.w8g8jt.c12.kafka.us-east-1.amazonaws.com:9098 --consumer.config client.properties --topic 0e2a0bfcc015.geo --from-beginning --group students

./kafka-console-consumer.sh --bootstrap-server b-3.pinterestmskcluster.w8g8jt.c12.kafka.us-east-1.amazonaws.com:9098,b-2.pinterestmskcluster.w8g8jt.c12.kafka.us-east-1.amazonaws.com:9098,b-1.pinterestmskcluster.w8g8jt.c12.kafka.us-east-1.amazonaws.com:9098 --consumer.config client.properties --topic 0e2a0bfcc015.geo --from-beginning --max-messages 10
```

START KAFKA REST BEFORE SENDING VIA API

./kafka-rest-start /home/ec2-user/Downlads/confluent-7.2.0/etc/kafka-rest/kafka-rest.properties

FROM THE confluent-7.2.0/bin